## General points

- Have some slides to follow, but the more discussion we can have the better 

- Have tried to go as light on the maths as possible 


## General points

- Have some slides to follow, but the more discussion we can have the better 

- Have tried to go as light on the maths as possible 

- Quite a lot of accompanying code beneath the slides, will make it public on github - caution there might be some bugs! 

![](figs/bug.webp){fig-align="center"}


## Why do we care?

- Interested in monitoring an aircraft wing, but only have access to data where we have sensors

- Sensors = cost 

::: {layout-ncol=2}

![](figs/aircraft%20wing/hawk.jpg)

![](figs/aircraft%20wing/hawk-sens.jpg)

:::


## Virtual sensing

- Interested in monitoring an aircraft wing, but only have access to data where we have sensors

- Sensors = cost 

- Use a Gaussian process to act as a "virtual sensor"

![](figs/aircraft%20wing/wing.png)


## Virtual sensing

- Interested in monitoring an aircraft wing, but can only get data where we have sensors

- Sensors = cost 

- Use a Gaussian process to act as a "virtual sensor"

![](figs/aircraft%20wing/wing-vs.png)

## Virtual sensing of different phenomena

- We can extend virtual sensing to using one type of measurement to predict another

- e.g predicting a wind turbine's power output from measured wind speed 


## Virtual sensing of different phenomena

::: {layout-ncol=2 layout-valign="center"}

![](figs/wind-turbine-small.jpg){width=450}

![](figs/powerCurve.png){width=450}

:::


## Many other reasons

- Surrogate modelling

- Dealing with complex behaviours that we don't have physical models for

- 'Digital twins'

## Commonality across tasks

- All revolve around learning a map from $X \rightarrow \mathbf{y}$

- In machine learning & statistics, we call this a regression problem


## Commonality across tasks

- We can use Gaussian process (regression) to learn this map!

- Boils down to fitting curves (functions) through data

- **First Gaussian process definition**; a tool for fitting curves through data

## Fitting curves


In [ ]:
from scipy.stats import norm
import matplotlib.pyplot as plt
import numpy as np 
from numpy.random import multivariate_normal
import random

import sys 
sys.path.append('../')

import kernels
import models

np.random.seed(20)
x = np.linspace(0,1,100)[:,None]

hyps = np.array([1,.1,1e-6])
kernel = kernels.SquaredExponential(x,x,hyps)
k, _ , _ = kernel.compute_kernel()

mu = np.zeros((100,))
nsamples = 1
y = multivariate_normal(mu, k,nsamples).T

npts = 10
random.seed(1)
randidx = random.sample(range(0,100),npts)

xsample = x[randidx]
ysample = y[randidx]

plt.plot(xsample,ysample,'kx')

## Fitting a GP


In [ ]:
D = (xsample,ysample)
mu_pred, cov_pred = models.predict(hyps,D,x,kernels.SquaredExponential)
var_pred = np.diag(cov_pred)

plt.figure()
plt.plot(xsample,ysample,'kx',label ='train pts')
plt.plot(x,mu_pred, label ='GP predictions')
plt.fill_between(x[:,0], mu_pred[:,0] - 3*np.sqrt(var_pred), mu_pred[:,0] + 3*np.sqrt(var_pred),color = 'darkgreen', alpha = 0.3, label ='confidence bounds')
plt.legend();

## Fitting a GP


In [ ]:
nsamples = 10
gp_samples = multivariate_normal(mu, cov_pred, nsamples).T
gp_samples += mu_pred

plt.figure()
plt.plot(x,gp_samples)
plt.plot(xsample,ysample,'kx',label ='train pts')
plt.title('Posterior GP samples')
plt.legend();


## So how do GPs do it?

- You have a friend that asks you how your beer tastes..

![](figs/hzy-ipa.webp){fig-align="center" width=100}


## So how do GPs do it?

::: {layout-ncol=2}

![](figs/hzy-ipa.webp){width=250}

![](figs/beer-recipe.jpg){width=350}


:::

## So how do GPs do it?

::: {layout-ncol=2}

![](figs/hzy-ipa.webp){width=250}

![](figs/hzy-ipa2.webp){width=450}


:::

## So how do GPs do it?

::: {layout-ncol=2}

![](figs/hzy-ipa.webp){width=250}

![](figs/guiness.jpg){width=300}


:::

## A second definition

Gaussian processes are a flexible tool for performing nonliner regression in a non-parametric, Bayesian manner

## The intersection between beer tasting and GPs

- GPs work in the same way! Use similarity between data points to tell us about others

## The intersection between beer tasting and GPs

- GPs work in the same way! Use similarity between data points to tell us about others

- Basis of this notion is treating each data point as a Gaussian distribution

- This mean we have some mean value, as well as some uncertainty on observed data 


## The Gaussian distribution 


In [ ]:
#initialize a normal distribution with frozen in mean=-1, std. dev.= 1
gaussian1 = norm(loc = 0., scale = 1.0)

x = np.arange(-10, 10, .1)

#plot the pdfs of these normal distributions 
plt.plot(x, gaussian1.pdf(x));
plt.ylabel('p(x)');
plt.xlabel('x');

$p(x) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp^{-\frac{(x-\mu)^2}{2\sigma^2}}$


## The Gaussian distribution


In [ ]:
gaussian2 = norm(loc = 0., scale = 1)
gaussian3 = norm(loc = 2, scale = 2.5)

plt.plot(x, gaussian2.pdf(x), x, gaussian3.pdf(x));
plt.ylabel('p(x)');
plt.xlabel('x');

$p(x) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp^{-\frac{(x-\mu)^2}{2\sigma^2}}$


## Measuring similarity between the data


In [ ]:
mu = np.array([0,0])
cov_ind = np.array([[1,0], [0,1]])
cov_cor = np.array([[1,0.85],[0.85,1]])

samples_ind = multivariate_normal(mu,cov_ind, size = 1000)
samples_cor = multivariate_normal(mu,cov_cor, size = 1000)

fig, axs = plt.subplots(1,2, figsize=(10,6))
axs[0].plot(samples_ind[:,0], samples_ind[:,1],'x', color = 'indigo')
axs[1].plot(samples_cor[:,0], samples_cor[:,1],'x', color = 'indigo')
plt.setp(axs, xlim=(-4,4), ylim=(-4,4));

fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axes
plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
plt.grid(False)
plt.xlabel("x")
plt.ylabel("y")

## Beyond 2 data points

d = 2


In [ ]:
np.random.seed(20)
x = np.linspace(0,1,100)[:,None]

hyps = np.array([1,.25])
kernel = kernels.SquaredExponential(x,x,hyps)
k, _ , _ = kernel.compute_kernel()

mu = np.zeros((100,))
nsamples = 5
y = multivariate_normal(mu, k,nsamples).T

d = 2
ysample = y[0::100//d,:]
index = np.linspace(1,d,d)

plt.plot(index,ysample)

## Beyond 2 data points

d = 4


In [ ]:
d = 4
ysample = y[0::100//d,:]
index = np.linspace(1,d,d)

plt.plot(index,ysample)

## Beyond 2 data points

d = 5


In [ ]:
d = 5
ysample = y[0::100//d,:]
index = np.linspace(1,d,d)

plt.plot(index,ysample)

## Beyond 2 data points

d = 10


In [ ]:
d = 10
ysample = y[0::100//d,:]
index = np.linspace(1,d,d)

plt.plot(index,ysample)

## Beyond 2 data points

d = 20


In [ ]:
d = 20
ysample = y[0::100//d,:]
index = np.linspace(1,d,d)

plt.plot(index,ysample)

## Beyond 2 data points

d = 50


In [ ]:
d = 50
ysample = y[0::100//d,:]
index = np.linspace(1,d,d)

plt.plot(index,ysample)

## A third & final definition

A Gaussian processes is a collection of random variables such that every subset of those random variables has a multivariate normal distribution


In [ ]:
d = 100
ysample = y[0::100//d,:]
index = np.linspace(1,d,d)

plt.plot(index,ysample)

## The core ingredients of a GP

- To construct a Gaussian process model, we need to define both 
a mean and a covariance function 

- $f(\mathbf{x}) \sim \mathcal{GP}( m(\mathbf{x}), k (\mathbf{x},\mathbf{x}'))$

## The core ingredients of a GP

- Mean can be viewed as an offset of predictions

- Covariance function controls how "closeness" of data points relates to their function value 

## The covariance function

- Will generally assume a zero mean function

- Many different types of kernel functions!

- Loosely, they govern the "shape" of functions that the GP will model

## Some examples 


In [ ]:
hyps = [1,0.5]

kernelSE = kernels.SquaredExponential(x,x,hyps)
kernelMA12 = kernels.Matern12(x,x,hyps)

kSE, _ , _ = kernelSE.compute_kernel()
kMA12, _ , _ = kernelMA12.compute_kernel()

mu = np.zeros((100,))
nsamples = 10

ySE = multivariate_normal(mu, kSE,nsamples).T
yMA12 = multivariate_normal(mu, kMA12,nsamples).T

fig, axs = plt.subplots(1,2, figsize=(10,6))
axs[0].plot(x,ySE)
axs[0].set_title('Samples from SE kernel');
axs[1].plot(x,yMA12)
axs[1].set_title('Samples from Matern 1/2 kernel');

## Some examples 


In [ ]:
hyps = [1,0.5]

kernelMA32 = kernels.Matern32(x,x,hyps)
kernelMA12 = kernels.Matern12(x,x,hyps)

kMA32, _ , _ = kernelMA32.compute_kernel()
kMA12, _ , _ = kernelMA12.compute_kernel()

mu = np.zeros((100,))
nsamples = 10

yMA12 = multivariate_normal(mu, kMA12,nsamples).T
yMA32 = multivariate_normal(mu, kMA32,nsamples).T

fig, axs = plt.subplots(1,2, figsize=(10,6))
axs[0].plot(x,yMA12)
axs[0].set_title('Samples from Matern 1/2 kernel');
axs[1].plot(x,yMA32)
axs[1].set_title('Samples from Matern 3/2 kernel');

## If we have more insight


In [ ]:
hyps = [1,0.5,0.5]

kernelPeriodic = kernels.Periodic(x,x,hyps)

kPeriodic, _ , _ = kernelPeriodic.compute_kernel()

mu = np.zeros((100,))
nsamples = 10

yPeriodic = multivariate_normal(mu, kPeriodic,nsamples).T

plt.figure()
plt.plot(x,yPeriodic)
plt.title('Samples from periodic kernel ');

## Breaking down the kernel

- $d = || \mathbf{x} - \mathbf{x}' ||$, where $||\mathbf{p},\mathbf{q}|| = \sqrt{\sum_i{(q_i - p_i)}^2}$


In [ ]:
# Define the two 2D points
x1, y1 = 2, 3
x2, y2 = 6, 7
eqx, eqy = 4,3.5

middle_x = (x1 + x2) / 2
middle_y = (y1 + y2) / 2
distance = ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5

# Plot the two points and the line connecting them
label_offset = 0.25
plt.plot([x1, x2], [y1, y2], 'go-',markersize = 8)
plt.text(x1 + 0.5*label_offset, y1 + 1.5*label_offset, f'Point 1 \n({x1},{y1})', ha='center')
plt.text(x2 - 1.8*label_offset, y2 - 1*label_offset, f'Point 2 \n({x2},{y2})', ha='center')

plt.text(middle_x, middle_y + 1.5*label_offset, f'd', ha='center', va='top')
plt.text(eqx, eqy, 'd = $\sqrt{(x_2 - x_1)^2 + (y_2 - y_1)^2}$', ha = 'left')

# Add labels and title
plt.xlabel('x');
plt.ylabel('y');

## Breaking down the kernel

- $d = || \mathbf{x} - \mathbf{x}' ||$, where $||\mathbf{p},\mathbf{q}|| = \sqrt{\sum_i{(q_i - p_i)}^2}$

<br/>

- $K_{SE}(\mathbf{x}, \mathbf{x}') = \sigma _f^2 \exp( - \frac{d^2}{2 l^2})$

<br/>

- $K_{MA12}(\mathbf{x}, \mathbf{x}') = \sigma _f^2 \exp( -\frac{d}{l})$

<br/>

- $K_{MA32}(\mathbf{x}, \mathbf{x}') = \sigma _f^2 ( 1+ \frac{\sqrt{3}d}{l})\exp( - \frac{\sqrt{3}d}{l})$

## Hyperparameters 

- For a given kernel, there will be some hyperparameters that needed to be selected 

- For both the Squared Expoential and Matern family of kernels, these are the lengthscale ($l$) and signal variance $(\sigma _f$)

## Hyperparameters 

- 

## How do we set hyperparameters?

- Generally won't know what these values should be...

## Summary 

1. Acquire training data $D =(X,\mathbf{y})$
2. Select mean and kernel (covariance function)  
    - if have prior knowledge, can put it in here!
3. Learn hyperparameters by training the model via minimisation of the *negative marginal log likelihood*
4. Make predictions by conditioning the model on the training data (shrinking possible functions where we have data)
5. Take over the world! 

## Draw backs of GPs

.. and there are several despite what you might think I believe after the last hour..

- Training & memory requirements - scale very badly as we increase number of training points
- Selecting a covariance function
- Hyperparameter learning can be challenging; local optima, for more complex kernel structures & if we try and recover full distributions over the parameters 

## Bayes rule for GPs

- Prior: $p(f) \sim \mathcal{N}(0,k(x,x'))$
- Likelihood $p(y|f,X) \sim \mathcal{N}(f,\sigma_n^2\mathbb{I})$ - data
- Marginal likelihood $p(y|X)$ - hyperparameter optimisation
- Posterior = Bayes rule = $p(f|y,X) = \frac{\mathcal{N}(0,k(x,x'))\mathcal{N}(f,\sigma_n^2\mathbb{I})}{p(y|X)}$
